In [2]:
# transform .simhess files to .h5 with cta-pipe for further analysis with DL1 data handler
# based on https://cta-observatory.github.io/ctapipe/tutorials/lst_analysis_bootcamp_2018.html
#import astropy.units as u
#from astropy.coordinates import SkyCoord, AltAz

import ctapipe
from ctapipe.containers import ImageParametersContainer
from ctapipe.io import EventSource

from ctapipe.calib import CameraCalibrator

from ctapipe.image import hillas_parameters, tailcuts_clean, morphology_parameters, leakage_parameters
from ctapipe.image import concentration_parameters

#from ctapipe.reco import HillasReconstructor
import numpy as np
from matplotlib import pyplot

#from copy import deepcopy
#import tempfile
import os
import h5py
import time
from datetime import timedelta

In [3]:
ctapipe.__version__

'0.12.0'

In [28]:
type2name_dict = {'LST':'HESS-II', 'MST':'HESS-I'}
def rename_cameras(source, type2name_dict):
    '''
    rename HESS cameras in the source derived from .simhess
    '''
    for t in source.subarray.tel.values():
        tel_type = t.type
        tel = type2name_dict.get(tel_type, 'HESS-I')
        t.camera.camera_name = tel
        t.camera.geometry.camera_name = tel
        t.type = tel
        t.name =tel
        t.optics.name = tel
    return 0

def cam_index2cam_type(source):
    """
        dictionary that translates camera index into camera type
        necessary for correct output of geometry tables from simhess to cta-pipe .h5
    """
    res = {}
    table = source.subarray.to_table()
    for i, ind in enumerate(table['camera_index']):
        if ind not in res:
            res[ind] = table['camera_type'][i]
    return res

def rename_hess_cameras(filename, source, path='/configuration/instrument/telescope/camera'):
    '''
    rename HESS cameras in the output .h5 file
    (move the corresponding subtables)
    '''
    h5_obj = h5py.File(filename, 'a')
    h5t = h5_obj[path]
    ind2type_dict = cam_index2cam_type(source)
    for ind, ctype in ind2type_dict.items():
        for key in h5t.keys():
            if key.find('_%s' % ind) > -1:
                key_new = key.replace('_%s' % ind, '_%s' % ctype)
                if key_new not in h5t.keys():
                    h5t.move(key, key_new)
    h5_obj.flush()
    h5_obj.close()
    return 0

def get_fn_out(folder_out, fn0, test=False):
    if fn0.startswith('/'):
        fn0 = fn0.split('/')[-1]
    if test:
        return folder_out + fn0.replace('.simhess.gz', '_test.h5')
    else:
        return folder_out + fn0.replace('.simhess.gz', '.h5')

In [16]:
test = False
#mode = 'phase2d'
#mode = 'phase2d'
#mode = 'phase2b5FC_electrons'
mode = 'phase2d_diff'
fns_in_sorted = None
if mode == 'HPC_test':
    test = True
    folder_in = '/home/saturn/caph/mpp228/CTA_data/Heidelberg_HESS_examples/'
    folder_out = folder_in[:-1] + '_h5/'
    fn0 = 'gamma_55deg_0deg_run42329___phase2d2_desert-ws0-crab-alt35-off1.0-pos2_cone5.simhess.gz'
    #fn0 = 'gamma_20deg_180deg_run1202___phase2d_simon3_desert-all_tel-_mix_atm_new_trig_thresh-sims-ref0.79_off0.5.simhess.gz'
    #fn0 = 'gamma_20deg_0deg_run8608___phase2d_simon3_desert-all_tel-correction_scheme_case_A_off0.5.simhess.gz'
    fns_in = [fn0]
elif mode == 'phase2d':
    alt = 70
    folder_in = '/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/\
phase2d/sim_telarray/phase2d/alt%i/' % alt
    folder_out = folder_in[:-1] + '_h5/'
    fns_in = os.listdir(folder_in)
    replace = False
elif mode == 'phase2b5FC_electrons':
    folder_in = '/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2b5FC/0.0deg/Data/'
    folder_out = folder_in[:-1] + '_h5/'
    fns_in = os.listdir(folder_in)
    replace = False
elif mode == 'phase2d_diff':
    fns_dict = {}
    ptypes = ['Proton', 'Electron', 'Gamma-diffuse']
    for ptype in ptypes:
        if ptype == 'Gamma-diffuse':
            folder_in = '/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/\
Desert/%s/20deg/180deg/0.0deg-ws0_cone5/Data/' % ptype
        else:
            folder_in = '/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/\
Desert/%s/20deg/180deg/0.0deg-ws0/Data/' % ptype
        fns_in = list(np.sort(os.listdir(folder_in)))
        fns_dict[ptype] = [(folder_in + fn) for fn in fns_in]
    fns_in = []
    i = 0
    nmax = np.max([len(fns_dict[ptype]) for ptype in ptypes])
    while i < nmax:
        for ptype in ptypes:
            if i  < len(fns_dict[ptype]):
                fns_in.append(fns_dict[ptype][i])
        i += 1
    fns_in_sorted = fns_in
    folder_out = '/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/\
Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/'
    replace = False
    
if not os.path.isdir(folder_out):
    os.makedirs(folder_out)

In [17]:
if 0:
    pkey = 'proton'
    if mode == 'phase2b5FC_electrons':
        gkey = 'electron'
    else:
        gkey = 'gamma'


    pfns_in = list(np.sort([fn for fn in fns_in if fn.find(pkey) > -1]))
    gfns_in = list(np.sort([fn for fn in fns_in if fn.find(gkey) > -1]))
    print(len(pfns_in), len(gfns_in), len(fns_in))

In [18]:
if 0:
    fn_check = [fn for fn in fns_in if fn not in pfns_in and fn not in gfns_in]
    print(fn_check)

In [20]:
if 0:
    nmax = min(len(pfns_in), len(gfns_in))
    fns_in_sorted = np.array([pfns_in[:nmax], gfns_in[:nmax]], dtype=str).T.flatten()
    if 0:
        # just in case for phase2d
        processed_ind = 618
        fns_in_sorted = fns_in_sorted[processed_ind:]
    print(fns_in_sorted[:6])
elif fns_in_sorted is None:
    fns_in_sorted = list(np.sort(fns_in))
if test:
    fns_in_sorted = fns_in_sorted[:2]


In [13]:
if mode == 'phase2d' and not test:
    fns_test = [fn.split('___')[1].replace('_cone5', '') for fn in fns_in_sorted]
    for i in range(100):
        if fns_test[2*i] != fns_test[2*i + 1]:
            print('Not identical condition in file %i' % i)
elif mode == 'phase2b5FC_electrons' and not test:
    fns_test = [fn.split('___')[1] for fn in fns_in_sorted]
    for i in range(100):
        if fns_test[2*i] != fns_test[2*i + 1]:
            print('Not identical condition in file %i' % i)

In [14]:
test

False

In [22]:
if 0:
    source = EventSource(fn_in, max_events=100)
    event = source[0]

In [29]:
t0 = time.time()
ntot = 0
sim_im_path = '/simulation/event/telescope/images'
im_path = 'dl1/event/telescope/images'

for k, fn_in in enumerate(fns_in_sorted):
    print('File %i out of %i' % (k + 1, len(fns_in_sorted)))
    if not fn_in0.startswith('/'):
        fn_in = folder_in + fn_in
    print(fn_in)
    fn_out = get_fn_out(folder_out, fn_in, test=test)
    print(fn_out)
    #if (not replace) and os.path.isfile(fn_out):
    #    os.remove(fn_in)
    #    continue
    if test:
        source = EventSource(fn_in, max_events=100)
    else:
        source = EventSource(fn_in)
    rename_cameras(source, type2name_dict)
    calib = CameraCalibrator(subarray=source.subarray)
    with ctapipe.io.DataWriter(event_source=source, output_path=fn_out, 
                               #write_stereo_shower=True, 
                               write_index_tables=True, 
                               write_mono_shower=True,
                               overwrite=True, write_parameters=True, write_images=True) as write_data:
        for i, event in enumerate(source):         
            try:
                calib(event)
                for tel_id, tel_data in event.dl1.tel.items():
                    #print(tel_id)
                    tel = source.subarray.tel[tel_id]
                    geom = tel.camera.geometry
                    image = tel_data.image
                    mask = tailcuts_clean(geom, image)
                    tel_data.image_mask = mask.astype(bool)
                    

                    event.dl1.tel[tel_id].parameters = ImageParametersContainer()
                    tel_data.parameters = ImageParametersContainer()
                    hillas_c = hillas_parameters(geom[mask], image[mask])
                    event.dl1.tel[tel_id].parameters.hillas = hillas_c
                    event.dl1.tel[tel_id].parameters.leakage = leakage_parameters(geom, image, mask)
                    event.dl1.tel[tel_id].parameters.morphology = morphology_parameters(geom, mask) 
                    event.dl1.tel[tel_id].parameters.concentration = concentration_parameters(geom, image, hillas_c)

                    # placeholder for the moment
                    event.simulation.tel[tel_id].true_parameters = ImageParametersContainer()
                    #event.simulation.tel[tel_id].parameters = ImageParametersContainer()
                    event.simulation.tel[tel_id].true_parameters.hillas = hillas_c

                write_data(event)
                ntot += 1
            except:
                pass
            if (i + 1) % 10 == 0 and mode == 'HPC_test':
                print('%i events' % (i + 1))
            elif (i + 1) % 1000 == 0 and mode != 'HPC_test':
                td = timedelta(seconds=(time.time() - t0))
                print('%i events, %i tot events, time since start: %s' % (i + 1, ntot, td))
                
        #with h5py.File(fn_out, 'a') as h5_obj:
        #    if sim_im_path not in h5_obj:
        #        h5_obj.create_group(sim_im_path)
                #for key in h5_obj[sim_im_path].keys():
                #    if not key.startswith('_i'):
                #        h5_obj[sim_im_path][key] = h5_obj[im_path][key]


        if test:
            print('write to file:')
            print(fn_out)
            
    try:
        rename_hess_cameras(fn_out, source, path='/configuration/instrument/telescope/camera')
    except:
        pass


File 1 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run13947___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run13947___phase2d2_desert-ws0-nsb1.00.h5
File 2 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Electron/20deg/180deg/0.0deg-ws0/Data/electron_20deg_180deg_run25444___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/electron_20deg_180deg_run25444___phase2d2_desert-ws0-nsb1.00.h5
File 3 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20

File 21 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run8950___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run8950___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 22 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20460___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20460___phase2d2_desert-ws0-nsb1.00.h5
File 23 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Electron/20deg/180deg/0.0deg-ws0/Data/e

File 41 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Electron/20deg/180deg/0.0deg-ws0/Data/electron_20deg_180deg_run25457___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/electron_20deg_180deg_run25457___phase2d2_desert-ws0-nsb1.00.h5
File 42 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run8957___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run8957___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 43 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Da

File 61 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20473___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20473___phase2d2_desert-ws0-nsb1.00.h5
File 62 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Electron/20deg/180deg/0.0deg-ws0/Data/electron_20deg_180deg_run25464___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/electron_20deg_180deg_run25464___phase2d2_desert-ws0-nsb1.00.h5
File 63 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma

File 81 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run8970___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run8970___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 82 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20480___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20480___phase2d2_desert-ws0-nsb1.00.h5
File 83 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Electron/20deg/180deg/0.0deg-ws0/Data/e

...

File 1101 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20903___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20903___phase2d2_desert-ws0-nsb1.00.h5
File 1102 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run9394___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run9394___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 1103 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Da

File 1121 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20913___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20913___phase2d2_desert-ws0-nsb1.00.h5
File 1122 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run9404___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run9404___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 1123 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Da

File 1141 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20923___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20923___phase2d2_desert-ws0-nsb1.00.h5
File 1142 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run9414___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run9414___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 1143 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Da

File 1161 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20933___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20933___phase2d2_desert-ws0-nsb1.00.h5
File 1162 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run9424___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run9424___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 1163 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Da

File 1181 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/proton_20deg_180deg_run20943___phase2d2_desert-ws0-nsb1.00.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/proton_20deg_180deg_run20943___phase2d2_desert-ws0-nsb1.00.h5
File 1182 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Gamma-diffuse/20deg/180deg/0.0deg-ws0_cone5/Data/gamma_20deg_180deg_run9434___phase2d2_desert-ws0-nsb1.00_cone5.simhess.gz
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton_Electron_Gamma-diffuse/20deg/180deg/0.0deg-ws0/Data_h5/gamma_20deg_180deg_run9434___phase2d2_desert-ws0-nsb1.00_cone5.h5
File 1183 out of 1200
/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Da

In [68]:
fn_out

'/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/Data_h5/proton_20deg_180deg_run20460___phase2d2_desert-ws0-nsb1.00.h5'

In [56]:
if 0:
    #remove the last file?
    os.remove(fn_out)

In [57]:
with h5py.File(fn_out, 'r') as h5_test:
    h5t = h5_test['/configuration/instrument/telescope/camera']
    print('Keys should contain HESS-I and HESS-II:')
    print(h5t.keys())


Keys should contain HESS-I and HESS-II:
<KeysViewHDF5 ['geometry_HESS-I', 'geometry_HESS-I.__table_column_meta__', 'geometry_HESS-II', 'geometry_HESS-II.__table_column_meta__', 'readout_HESS-I', 'readout_HESS-I.__table_column_meta__', 'readout_HESS-II', 'readout_HESS-II.__table_column_meta__']>


In [58]:
folder_out

'/home/saturn/caph/mpp228/HESS_data/HESS_data_MC/sim_telarray/phase2d/NSB1.00/Desert/Proton/20deg/180deg/0.0deg-ws0/Data/Data_h5/'